In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [4]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [19]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [91]:

nActions = 4
tableSize = 4
hiddenFeatures = 16
# temporalWindow = 5
temporalWindow = 1
env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))
model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha
# model.add(Activation(activation="elu")) # Tune alpha

model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()
replayMemory = SequentialMemory(limit=100, window_length=temporalWindow)
dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=1000,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=20000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 20000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   500/20000: episode: 1, duration: 0.798s, episode steps: 500, steps per second: 627, episode reward: -19310.000, mean reward: -38.620 [-50.000, 88.000], mean action: 1.860 [0.000, 3.000],  loss: --, mae: --, mean_q: --


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1089/20000: episode: 2, duration: 2.343s, episode steps: 589, steps per second: 251, episode reward: -22420.000, mean reward: -38.065 [-50.000, 128.000], mean action: 1.438 [0.000, 3.000],  loss: 684.166446, mae: 19.674812, mean_q: 7.476380
  1533/20000: episode: 3, duration: 2.261s, episode steps: 444, steps per second: 196, episode reward: -13736.000, mean reward: -30.937 [-50.000, 128.000], mean action: 1.628 [0.000, 3.000],  loss: 418.702454, mae: 34.614090, mean_q: -26.742210
  1868/20000: episode: 4, duration: 1.710s, episode steps: 335, steps per second: 196, episode reward: -8270.000, mean reward: -24.687 [-50.000, 128.000], mean action: 1.072 [0.000, 3.000],  loss: 445.915771, mae: 57.482182, mean_q: -56.674091
  1965/20000: episode: 5, duration: 0.486s, episode steps:  97, steps per second: 200, episode reward: -1310.000, mean reward: -13.505 [-50.000, 76.000], mean action: 2.361 [0.000, 3.000],  loss: 362.823425, mae: 58.967587, mean_q: -58.463760
  2382/20000: episode: 6,

# Simulate Game

In [95]:
# Play the game with predicts

def log2_transform(grid):
    transformed = np.where(grid > 0, np.log2(grid), 0)
    return transformed

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(10)]
metrics = {
    "gameId": [],
    "score": [],
    "higherTile": [],
    "invalidMove (%)": [],
    "steps": [],
    "upMoves": [],
    "downMoves": [],
    "leftMoves": [],
    "rightMoves": [],
    "upSeqAvg": [],
    "downSeqAvg": [],
    "leftSeqAvg": [],
    "rightSeqAvg": [],
}
for i, game in enumerate(games):
    # Metrics
    steps = 0
    invalidMove = 0
    upMoves = 0
    downMoves = 0
    leftMoves = 0
    rightMoves = 0
    upSeqAvg = 0
    downSeqAvg = 0
    leftSeqAvg = 0
    rightSeqAvg = 0

    sequences = {
        "up": [],
        "down": [],
        "left": [],
        "right": []
    }
    lastMove = None

    # print(f"Game {i + 1} initial state:")
    # game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        observation = log2_transform(observation)
        predictedAction = dqn.forward(observation)
        
        # print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        if not game.move(actionMapping[predictedAction]):
            invalidMove += 1
        
        # print(f"Game {i + 1} currrent state:")
        # game.render()

        # input("\nPress any key to continue...")

        steps += 1
        if actionMapping[predictedAction] == "up":
            upMoves += 1
            if lastMove == "up":
                sequences["up"][-1] += 1
            else:
                sequences["up"].append(1)
        elif actionMapping[predictedAction] == "down":
            downMoves += 1
            if lastMove == "down":
                sequences["down"][-1] += 1
            else:
                sequences["down"].append(1)
        elif actionMapping[predictedAction] == "left":
            leftMoves += 1
            if lastMove == "left":
                sequences["left"][-1] += 1
            else:
                sequences["left"].append(1)
        elif actionMapping[predictedAction] == "right":
            rightMoves += 1
            if lastMove == "right":
                sequences["right"][-1] += 1
            else:
                sequences["right"].append(1)
        
        lastMove = actionMapping[predictedAction]

    print(f"Game {i + 1} is over.")

    metrics["gameId"].append(i)
    metrics["score"].append(game.score)
    metrics["higherTile"].append(np.max(game.grid))
    metrics["invalidMove (%)"].append(invalidMove/steps)
    metrics["steps"].append(steps)
    metrics["upMoves"].append(upMoves)
    metrics["downMoves"].append(downMoves)
    metrics["leftMoves"].append(leftMoves)
    metrics["rightMoves"].append(rightMoves)
    metrics["upSeqAvg"].append(np.mean(np.array(sequences["up"])))
    metrics["downSeqAvg"].append(np.mean(np.array(sequences["down"])))
    metrics["leftSeqAvg"].append(np.mean(np.array(sequences["left"])))
    metrics["rightSeqAvg"].append(np.mean(np.array(sequences["right"])))


metrics = pd.DataFrame(metrics).set_index("gameId")
display(metrics)
display(metrics.describe())


/tmp/ipykernel_1002/845857735.py:4: RuntimeWarning: divide by zero encountered in log2
  transformed = np.where(grid > 0, np.log2(grid), 0)


Game 1 is over.
Game 2 is over.
Game 3 is over.
Game 4 is over.
Game 5 is over.
Game 6 is over.
Game 7 is over.
Game 8 is over.
Game 9 is over.
Game 10 is over.


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
gameId,,,,,,,,,,,,
0,688,64,0.500000,182,8,8,122,44,1.000000,1.000000,3.297297,1.257143
1,1008,128,0.430851,188,16,14,122,36,1.142857,1.000000,2.711111,1.000000
2,1252,128,0.519435,283,21,17,182,63,1.105263,1.000000,3.370370,1.312500
3,272,32,0.242857,70,8,5,29,28,1.000000,1.250000,1.318182,1.473684
4,708,64,0.675862,290,15,12,196,67,1.000000,1.000000,5.600000,2.030303
5,756,64,0.672026,311,19,11,247,34,1.187500,1.100000,6.024390,1.030303
6,700,64,0.449102,167,20,12,106,29,1.176471,1.000000,3.212121,1.074074
7,392,32,0.535714,140,55,12,60,13,4.230769,1.090909,3.000000,1.000000
8,336,32,0.132353,68,16,11,27,14,1.142857,1.000000,1.588235,1.272727


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,727.200000,73.600000,0.449510,188.600000,21.800000,11.50000,116.600000,38.700000,1.548572,1.052424,3.175214,1.285550
std,336.263389,40.053298,0.173827,85.414285,14.680297,3.24037,73.327576,19.264533,1.043326,0.081803,1.586444,0.312501
min,272.000000,32.000000,0.132353,68.000000,8.000000,5.00000,27.000000,13.000000,1.000000,1.000000,1.318182,1.000000
25%,466.000000,40.000000,0.360387,146.750000,15.250000,11.00000,63.750000,28.250000,1.026316,1.000000,1.900604,1.041246
50%,704.000000,64.000000,0.474551,184.500000,17.500000,12.00000,114.000000,35.000000,1.142857,1.000000,3.106061,1.264935
75%,945.000000,112.000000,0.531644,259.250000,20.750000,12.75000,167.000000,55.250000,1.184743,1.089015,3.352102,1.381696
max,1252.000000,128.000000,0.675862,311.000000,55.000000,17.00000,247.000000,67.000000,4.230769,1.250000,6.024390,2.030303


# Manual Game

In [ ]:
moves = ["up", "left", "right", "down"]
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


